In [1]:
import json
import os

def concat_json_files(folder_path):
    combined_data = []

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                data = json.load(file)
                combined_data.extend(data)

    return combined_data

# Path to the folder containing JSON files
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS_300/NON_MERGE'

# Concatenate the JSON data
combined_data = concat_json_files(folder_path)

# Write the combined data to a new JSON file
with open('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FD_150_W.json', 'w') as outfile:
    json.dump(combined_data, outfile, indent=4)


In [2]:
import os
import pandas as pd

def list_files(directory, extension):
    files = []
    # Walk through the directory
    for root, dirs, files_in_dir in os.walk(directory):
        for file in files_in_dir:
            if file.endswith(extension):
                full_path = os.path.join(root, file)
                files.append(full_path)
    return files

# Directory containing the files
directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS_300'

# List all '.fk_results.dat' files
file_paths = list_files(directory, '.fk_results.dat')

# Create a DataFrame from the file paths
df = pd.DataFrame(file_paths, columns=['File Path'])

# Sort the DataFrame alphabetically
sorted_df = df.sort_values(by='File Path')

# Display the sorted DataFrame
print(sorted_df)


                                             File Path
20   /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
60   /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
48   /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
288  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
320  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
..                                                 ...
189  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
172  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
136  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
201  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
227  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...

[435 rows x 1 columns]


In [4]:
sorted_df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/notebook/FK_Database.csv', index=False)

In [11]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/features/I06H1_TEST_Dataset.csv')

# Remove the 'cluster' column
df1 = df1.drop(columns=['cluster'])

# Load the second CSV file
df2 = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/Test_Segments_Timestamp.csv')

# Merge the two dataframes on 'segment_id'
merged_df = pd.merge(df1, df2, on='segment_id')

# Save the merged dataframe to a new CSV file
merged_df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/features/I06H1_TEST_Dataset.csv', index=False)

print("Merging completed. The output is saved as 'merged_output.csv'.")


Merging completed. The output is saved as 'merged_output.csv'.


In [17]:
# Load the CSV file into a DataFrame
data = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/Test_Train_Segments.csv')

# Sort the DataFrame based on the 'Starttime' column
data['Start_Time'] = pd.to_datetime(data['Start_Time'])  # Convert 'Starttime' to datetime if it's not already
data.sort_values(by='Start_Time', inplace=True)

# Save the sorted data to a new CSV file
data.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/Test_Train_Segments.csv', index=False)

In [ ]:
import pandas as pd

# Load the main CSV file
main_df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/features/TEST_Dataset.csv')

# Load the second CSV file containing start and end times
time_df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/Test.csv')

# Convert the 'Start_Time' and 'End_Time' columns to datetime in the second DataFrame
time_df['Start_Time'] = pd.to_datetime(time_df['Start_Time'])
time_df['End_Time'] = pd.to_datetime(time_df['End_Time'])

# Calculate the midpoint for each time span in the second DataFrame
time_df['Timestamp'] = time_df['Start_Time'] + (time_df['End_Time'] - time_df['Start_Time']) / 2

# Merge the midpoint data into the main DataFrame based on 'segment_id'
merged_df = pd.merge(main_df, time_df[['segment_id', 'Timestamp']], on='segment_id', how='left')

# Now 'merged_df' contains your main data along with the 'Midpoint' column
print(merged_df)


In [8]:
merged_df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/features/TEST_Dataset.csv',index=False)

In [1]:
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor

def process_file(file_path):
    # Read each file and sort
    df = pd.read_csv(file_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    return df.sort_values(by='Datetime')

def read_csvs_from_folder(folder):
    files = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.csv')]
    with ProcessPoolExecutor() as executor:
        # Use parallel processing to read and sort files
        dataframes = list(executor.map(process_file, files))
    return dataframes

# Paths to your folders
folder1 = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/test'
folder2 = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/train'

# Read and combine all CSV files from both folders using parallel processing
df1 = read_csvs_from_folder(folder1)
df2 = read_csvs_from_folder(folder2)

# Concatenate all DataFrames and sort
combined_df = pd.concat(df1 + df2, ignore_index=True).sort_values(by='Datetime')

# Now 'combined_df' contains all your data, sorted by 'Datetime'
print(combined_df)


                          Datetime     I06H1     I06H2     I06H3     I06H4  \
3313200 2018-07-01 03:00:09.350000  0.656045  0.368027  0.421031  0.755403   
3313201 2018-07-01 03:00:09.400000  0.611502  0.327639  0.364624  0.898304   
3313202 2018-07-01 03:00:09.450000  0.497063  0.250486  0.453283  0.990035   
3313203 2018-07-01 03:00:09.500000  0.279066  0.262781  0.664350  1.000000   
3313204 2018-07-01 03:00:09.550000  0.053899  0.441433  0.791544  0.965469   
...                            ...       ...       ...       ...       ...   
24495   2019-08-10 21:10:06.949999  0.503063  0.055187  0.698695  0.843839   
24496   2019-08-10 21:10:06.999999  0.578968  0.149821  0.521312  0.568478   
24497   2019-08-10 21:10:07.049999  0.640236  0.236989  0.371320  0.371196   
24498   2019-08-10 21:10:07.099999  0.706414  0.281897  0.348975  0.436399   
24499   2019-08-10 21:10:07.149999  0.780988  0.308713  0.379025  0.589302   

            I06H6     I06H7     I06H8  
3313200  0.443750  0.36

In [1]:
import pandas as pd
import glob

# Path to the folder containing the CSV files
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_CSV_NEW'  # Replace with the path to your folder

# Get a list of all CSV files in the folder
csv_files = glob.glob(f"{folder_path}/*.csv")

# Load each CSV file and append it to the list
all_dataframes = []
for file in csv_files:
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    all_dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(all_dataframes)

# Sort the combined DataFrame by the index (date)
combined_df.sort_index(inplace=True)

# Save the combined and sorted DataFrame to a new CSV file
output_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_WAVEFORM_ARRAY.csv'  # Replace with your desired output path
combined_df.to_csv(output_path)

print(f"Combined CSV saved to {output_path}")


: 

In [2]:
combined_df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/TRAIN_TEST_Dataset.csv',index=False)

In [2]:
import pandas as pd
df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/TRAIN_TEST_TS_Dataset.csv')

In [6]:
# Sort the DataFrame by the datetime column
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by='Datetime', inplace=True)

# Define the percentage split for training and testing data
train_ratio = 0.8  # You can adjust this ratio as needed

# Calculate the split index
split_index = int(train_ratio * len(df))

# Split the data into training and testing sets
train_data = df[:split_index]
test_data = df[split_index:]

# Display the shapes of the training and testing sets
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)


Training data shape: (17725440, 8)
Testing data shape: (4431360, 8)


In [ ]:
import pandas as pd

# Load the two CSV files into DataFrames
data1 = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/Test_Segments_Timestamp.csv')
data2 = pd.read_csv('data2.csv')

# Merge data1 and data2 based on the 'segment_id' column
merged_data = pd.merge(data1, data2[['segment_id', 'Timestamp']], on='segment_id', how='left')

# Save the merged data to a new CSV file
merged_data.to_csv('merged_data.csv', index=False)


In [12]:
import pandas as pd

# Load the VONA_Filtered CSV file
vona_filtered_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/VONA_Filtered.csv'  # Replace with your file path
vona_filtered_df = pd.read_csv(vona_filtered_path)

# Convert 'Issued' to datetime and rename the column to 'Starttime'
vona_filtered_df['Starttime'] = pd.to_datetime(vona_filtered_df['Issued'])

# Drop the original 'Issued' column
vona_filtered_df.drop('Issued', axis=1, inplace=True)

# Filter out rows where 'Maximum Duration (Seconds)' is missing
vona_filtered_df = vona_filtered_df[vona_filtered_df['Maximum Duration (Seconds)'].notna()]

# Calculate 'Endtime' by adding 'Maximum Duration (Seconds)' to 'Starttime'
vona_filtered_df['Endtime'] = vona_filtered_df['Starttime'] + pd.to_timedelta(vona_filtered_df['Maximum Duration (Seconds)'], unit='s')

# Display the modified DataFrame
print(vona_filtered_df.head())


  Notice Number  Volcanic Cloud Height (M)  Maximum Duration (Seconds)  \
1    2023KRA065                     1157.0                       232.0   
2    2023KRA064                     2157.0                       182.0   
3    2023KRA063                     1657.0                        59.0   
4    2023KRA062                      657.0                        73.0   
5    2023KRA061                     1657.0                       182.0   

                  Starttime                   Endtime  
1 2023-09-10 05:36:00+00:00 2023-09-10 05:39:52+00:00  
2 2023-07-20 01:52:00+00:00 2023-07-20 01:55:02+00:00  
3 2023-07-20 01:51:00+00:00 2023-07-20 01:51:59+00:00  
4 2023-07-20 01:43:00+00:00 2023-07-20 01:44:13+00:00  
5 2023-06-19 01:22:00+00:00 2023-06-19 01:25:02+00:00  


In [14]:
import pandas as pd

# Load the data
test_train_segments_df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/Test_Train_Segments.csv')
vona_filtered_df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/VONA_Filtered.csv')  # This is the preprocessed file

# Convert 'Starttime' and 'Endtime' to datetime in both dataframes
test_train_segments_df['Starttime'] = pd.to_datetime(test_train_segments_df['Starttime'], errors='coerce')
test_train_segments_df['Endtime'] = pd.to_datetime(test_train_segments_df['Endtime'], errors='coerce')
vona_filtered_df['Starttime'] = pd.to_datetime(vona_filtered_df['Starttime'], errors='coerce')
vona_filtered_df['Endtime'] = pd.to_datetime(vona_filtered_df['Endtime'], errors='coerce')

# Initialize a label column
test_train_segments_df['label'] = 'No Event'

# Label each segment based on overlap with events
for _, event in vona_filtered_df.iterrows():
    event_start = event['Starttime']
    event_end = event['Endtime']
    if pd.notna(event_start) and pd.notna(event_end):
        mask = ((test_train_segments_df['Starttime'] < event_end) & (test_train_segments_df['Endtime'] > event_start))
        test_train_segments_df.loc[mask, 'label'] = 'Event'

# Display the result
test_train_segments_df.head()


TypeError: Invalid comparison between dtype=datetime64[ns] and Timestamp

In [1]:
import pandas as pd
df_train_test = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/timeseries/TRAIN_TEST_TS_Dataset.csv')
df_train_test = df_train_test.drop(['I06H2','I06H3','I06H4','I06H6','I06H7', 'I06H8'], axis=1)
df_train_test.head()

,Datetime,I06H1
0,2018-07-01 03:00:09.350,0.656045
1,2018-07-01 03:00:09.400,0.611502
2,2018-07-01 03:00:09.450,0.497063
3,2018-07-01 03:00:09.500,0.279066
4,2018-07-01 03:00:09.550,0.053899


In [4]:
print(df_train_test.shape)

(22156800, 2)


In [1]:
import pandas as pd
import os
import glob

# Define the folder path that contains your CSV files
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/Features_Real_test'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# List to hold dataframes
dfs = []

# Loop over the list of csv files
for file in csv_files:
    # Read the csv file
    df = pd.read_csv(file)
    
    # Parse the start_time column as datetime and set as index
    df['start_time'] = pd.to_datetime(df['start_time'])
    df = df.set_index('start_time')
    
    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(dfs)

# Sort by index (start_time)
combined_df = combined_df.sort_index()

# Optionally, you can save the combined dataframe to a new csv file
combined_df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/Real_test_data.csv')


In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/hello.csv')
array = np.array(df.index)
print(array)




['2018-06-24 22:50:00+00:00' '2018-06-24 22:51:15+00:00'
 '2018-06-25 20:30:17.200000+00:00' ... '2019-01-31 18:16:46.100000+00:00'
 '2019-01-31 19:25:31.100000+00:00' '2019-01-31 19:26:46.100000+00:00']


In [17]:
import pandas as pd
import numpy as np

# Read the CSV file into a DataFrame
df = pd.read_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/Real_test_data.csv')

# Convert the "start_time" column to a numpy array of UTC datetime objects
start_time_array = pd.to_datetime(df['start_time']).to_numpy()
print(start_time_array)

# Save the numpy array as a .npy file
np.save('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/Test_Datetime_Array.npy', start_time_array)

[Timestamp('2019-02-01 13:25:00+0000', tz='UTC')
 Timestamp('2019-02-01 13:26:15+0000', tz='UTC')
 Timestamp('2019-02-01 14:45:00+0000', tz='UTC')
 Timestamp('2019-02-01 14:46:15+0000', tz='UTC')
 Timestamp('2019-02-02 00:10:19.900000+0000', tz='UTC')
 Timestamp('2019-02-02 00:11:34.900000+0000', tz='UTC')
 Timestamp('2019-02-02 00:12:49.900000+0000', tz='UTC')
 Timestamp('2019-02-02 00:14:04.900000+0000', tz='UTC')
 Timestamp('2019-02-02 04:20:19.900000+0000', tz='UTC')
 Timestamp('2019-02-02 04:21:34.900000+0000', tz='UTC')
 Timestamp('2019-02-02 04:22:49.900000+0000', tz='UTC')
 Timestamp('2019-02-02 04:24:04.900000+0000', tz='UTC')
 Timestamp('2019-02-02 05:12:49.900000+0000', tz='UTC')
 Timestamp('2019-02-02 05:14:04.900000+0000', tz='UTC')
 Timestamp('2019-02-02 09:22:49.900000+0000', tz='UTC')
 Timestamp('2019-02-02 09:24:04.900000+0000', tz='UTC')
 Timestamp('2019-02-02 11:25:19.900000+0000', tz='UTC')
 Timestamp('2019-02-02 11:26:34.900000+0000', tz='UTC')
 Timestamp('2019-02-

TypeError: 'tuple' object is not callable

In [1]:
# Let's load the necessary libraries and the provided data
import numpy as np
import pandas as pd

# Load the numpy array containing the datetime index
datetime_index = np.load('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/datetimeindex2.npy')
datetime_index = pd.to_datetime(datetime_index)  # Convert to pandas datetime index

# Load the cluster labels
y = np.loadtxt('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/Km-n3-ft_Rewritten.txt')

# Combine the datetime index and the cluster labels into a DataFrame
df = pd.DataFrame({'DateTime': datetime_index, 'Cluster': y.astype(int)})

df.head()  # Show the first few rows of the DataFrame


,DateTime,Cluster
0,2018-06-24 22:50:00.000,1
1,2018-06-24 22:51:15.000,0
2,2018-06-25 20:30:17.200,0
3,2018-06-25 20:31:32.200,0
4,2018-06-26 02:50:32.150,0


In [3]:
df.to_csv('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed2_stft/clustered_datetime.csv', index=False)